<a href="https://colab.research.google.com/github/HaywhyCoder/imdb-lstm/blob/main/imdb-lstm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### Import Libraries

In [ ]:
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM
from tensorflow.keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from bs4 import BeautifulSoup
import re
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

In [ ]:
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

### Preprocess Text Data

In [ ]:
dataset = pd.read_csv("/content/drive/MyDrive/IMDB Dataset - IMDB Dataset.csv")
data = dataset.copy()
data.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [ ]:
data['sentiment'].value_counts()

In [ ]:
def clean_data(row):
  lemmatizer = WordNetLemmatizer()
  sentence = row[0]
  review = BeautifulSoup(sentence).get_text()
  clean_sentence = re.sub(r'[^A-Za-z\s]+', '', review.lower())
  stops = set(stopwords.words('english'))
  tokens = [lemmatizer.lemmatize(word) for word in clean_sentence.split() if word not in stops]
  row[0] = ' '.join(tokens)
  return row

In [ ]:
cleaned_data = data.apply(clean_data, axis=1)

<ipython-input-16-428ea69da65d>:3: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  sentence = row[0]
<ipython-input-16-428ea69da65d>:8: FutureWarning: Series.__setitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To set a value by position, use `ser.iloc[pos] = value`
  row[0] = ' '.join(tokens)


In [ ]:
cleaned_data.head()

,review,sentiment
0,one reviewer mentioned watching oz episode you...,positive
1,wonderful little production filming technique ...,positive
2,thought wonderful way spend time hot summer we...,positive
3,basically there family little boy jake think t...,negative
4,petter matteis love time money visually stunni...,positive


In [ ]:
cleaned_data.map({'positive': 1, 'negative': 0})
cleaned_data.head()

<ipython-input-20-e9d3bcf61a7f>:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  cleaned_data.replace({'sentiment' : {'positive': 1, 'negative': 0}}, inplace=True)


,review,sentiment
0,one reviewer mentioned watching oz episode you...,1
1,wonderful little production filming technique ...,1
2,thought wonderful way spend time hot summer we...,1
3,basically there family little boy jake think t...,0
4,petter matteis love time money visually stunni...,1


In [ ]:
# Split data into train and test sets
train_data, test_data = train_test_split(cleaned_data, test_size=0.2, random_state=42)
print(train_data.shape)
print(test_data.shape)

(40000, 2)
(10000, 2)


#### Data Preprocessing

In [ ]:
#Tokenize test data
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(train_data['review'])
X_train = pad_sequences(tokenizer.texts_to_sequences(train_data['review']), maxlen=200)
X_test = pad_sequences(tokenizer.texts_to_sequences(test_data['review']), maxlen=200)

In [ ]:
print(X_train)

[[   0    0    0 ...  121  129 3450]
 [   0    0    0 ...  118   24   33]
 [   0    0    0 ...  212  614    8]
 ...
 [   0    0    0 ...  999  364  521]
 [   0    0    0 ... 1141  153   33]
 [   0    0    0 ...  316   16 1780]]


In [ ]:
Y_train = train_data['sentiment']
Y_test = test_data['sentiment']

#### Build the LSTM Model

In [ ]:
model = Sequential()
model.add(Embedding(input_dim=5000, output_dim=128, input_length=200))
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [ ]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [ ]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
model.fit(X_train, Y_train, epochs=5, batch_size=64, validation_split=0.2)

Epoch 1/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 213s 411ms/step - accuracy: 0.7657 - loss: 0.4773 - val_accuracy: 0.8634 - val_loss: 0.3282
Epoch 2/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 252s 399ms/step - accuracy: 0.8796 - loss: 0.3014 - val_accuracy: 0.8673 - val_loss: 0.3175
Epoch 3/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 208s 411ms/step - accuracy: 0.9083 - loss: 0.2412 - val_accuracy: 0.8570 - val_loss: 0.3370
Epoch 4/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 254s 395ms/step - accuracy: 0.9188 - loss: 0.2075 - val_accuracy: 0.8699 - val_loss: 0.3315
Epoch 5/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 203s 406ms/step - accuracy: 0.9340 - loss: 0.1754 - val_accuracy: 0.8659 - val_loss: 0.3484


In [ ]:
loss, accuracy = model.evaluate(X_test, Y_test)
print(f"Test Loss: {loss}")
print(f'Test Accuracy: {accuracy}')

313/313 ━━━━━━━━━━━━━━━━━━━━ 42s 132ms/step - accuracy: 0.8677 - loss: 0.3392
Test Loss: 0.3384304940700531
Test Accuracy: 0.8708000183105469


In [ ]:
# Predict and evaluate using confusion matrix
Y_pred = (model.predict(X_test) > 0.5).astype(int)
cm = confusion_matrix(Y_test, Y_pred)
print("Confusion Matrix:\n", cm)

313/313 ━━━━━━━━━━━━━━━━━━━━ 45s 143ms/step
Confusion Matrix:
 [[4331  630]
 [ 662 4377]]


In [ ]:
# Performance metrics
report = classification_report(Y_test, Y_pred, target_names=['Negative', 'Positive'])
print("Classification Report:\n", report)

Classification Report:
               precision    recall  f1-score   support

    Negative       0.87      0.87      0.87      4961
    Positive       0.87      0.87      0.87      5039

    accuracy                           0.87     10000
   macro avg       0.87      0.87      0.87     10000
weighted avg       0.87      0.87      0.87     10000

